In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_path = '../data/hy_round1_train_20200102'
test_path = '../data/hy_round1_testA_20200102'

train_df_list = []
for file_name in os.listdir(train_path):
    df = pd.read_csv(os.path.join(train_path, file_name))
    train_df_list.append(df)
    
test_df_list = []
for file_name in os.listdir(test_path):
    df = pd.read_csv(os.path.join(test_path, file_name))
    test_df_list.append(df)

train_df = pd.concat(train_df_list)
test_df = pd.concat(test_df_list)

train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')
test_df['time'] = pd.to_datetime(test_df['time'], format='%m%d %H:%M:%S')

all_df = pd.concat([train_df, test_df], sort=False)

In [3]:
v = []
for value in all_df['速度'].values:
    if value <= 3:
        v.append(0)
    elif value <= 14:
        v.append(1)
    elif value <= 23:
        v.append(2)
    else:
        v.append(3)
all_df['v'] = v

In [4]:
d = []
for value in all_df['方向'].values:
    if value > 337.5 or value <= 22.5:
        d.append(0)
    elif value <= 67.5:
        d.append(1)
    elif value <= 112.5:
        d.append(2)
    elif value <= 157.5:
        d.append(3)
    elif value <= 202.5:
        d.append(4)
    elif value <= 247.5:
        d.append(5)
    elif value <= 292.5:
        d.append(6)
    elif value <= 337.5:
        d.append(7)
all_df['d'] = d

In [5]:
cross_features = []
for ship_id, group in all_df.groupby('渔船ID'):
    group = group.sort_values(by=['time'])
    cross_features.append([])
#     for i in range(4):
#         temp = group[group['v'] == i]
#         for j in range(8):
#             if temp.shape[0] > 0:
#                 cross_features[-1].append(temp[temp['d'] == j].shape[0] / temp.shape[0])
#             else:
#                 cross_features[-1].append(0)
#     for i in range(8):
#         temp = group[group['d'] == i]
#         for j in range(4):
#             if temp.shape[0] > 0:
#                 cross_features[-1].append(temp[temp['v'] == j].shape[0] / temp.shape[0])
#             else:
#                 cross_features[-1].append(0)
#     for i in range(4):
#         temp = group[group['v'] == i]
#         if temp.shape[0] == 0:
#             cross_features[-1].append(0)
#             cross_features[-1].append(0)
#             cross_features[-1].append(0)
#             cross_features[-1].append(0)
#         else:
#             x = temp['x'].values
#             y = temp['y'].values
#             cross_features[-1].append(np.min(x))
#             cross_features[-1].append(np.max(x))
#             cross_features[-1].append(np.min(y))
#             cross_features[-1].append(np.max(y))
    for i in range(8):
        temp = group[group['d'] == i]
        if temp.shape[0] == 0:
            cross_features[-1].append(0)
            cross_features[-1].append(0)
            cross_features[-1].append(0)
            cross_features[-1].append(0)
        else:
            x = temp['x'].values
            y = temp['y'].values
            cross_features[-1].append(np.min(x))
            cross_features[-1].append(np.max(x))
            cross_features[-1].append(np.min(y))
            cross_features[-1].append(np.max(y))

In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

import os
import lightgbm as lgb
from sklearn import preprocessing

In [7]:
def get_model():
    exported_pipeline = make_pipeline(
        RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.7000000000000001, n_estimators=100), step=0.1),
        StandardScaler(),
        StackingEstimator(estimator=SGDClassifier(alpha=0.001, eta0=0.01, fit_intercept=False, l1_ratio=1.0, learning_rate="invscaling", loss="perceptron", penalty="elasticnet", power_t=0.5)),
        GradientBoostingClassifier(learning_rate=0.5, max_depth=7, max_features=0.15000000000000002, min_samples_leaf=2, min_samples_split=2, n_estimators=100, subsample=0.8500000000000001)
    )
    set_param_recursive(exported_pipeline.steps, 'random_state', 2020)
    return exported_pipeline

In [8]:
def feature_generate_manually():
    train_path = '../data/hy_round1_train_20200102'
    test_path = '../data/hy_round1_testA_20200102'

    train_df_list = []
    for file_name in os.listdir(train_path):
        df = pd.read_csv(os.path.join(train_path, file_name))
        train_df_list.append(df)

    test_df_list = []
    for file_name in os.listdir(test_path):
        df = pd.read_csv(os.path.join(test_path, file_name))
        test_df_list.append(df)

    train_df = pd.concat(train_df_list)
    test_df = pd.concat(test_df_list)

    train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')
    test_df['time'] = pd.to_datetime(test_df['time'], format='%m%d %H:%M:%S')

    all_df = pd.concat([train_df, test_df], sort=False)

    new_df = all_df.groupby('渔船ID').agg(x_min=('x', 'min'), x_max=('x', 'max'), x_mean=('x', 'mean'), x_std=('x', 'std'), x_skew=('x', 'skew'), x_sum=('x', 'sum'),
                y_min=('y', 'min'), y_max=('y', 'max'), y_mean=('y', 'mean'), y_std=('y', 'std'), y_skew=('y', 'skew'), y_sum=('y', 'sum'),
                v_min=('速度', 'min'), v_max=('速度', 'max'), v_mean=('速度', 'mean'), v_std=('速度', 'std'), v_skew=('速度', 'skew'), v_sum=('速度', 'sum'),
                d_min=('方向', 'min'), d_max=('方向', 'max'), d_mean=('方向', 'mean'), d_std=('方向', 'std'), d_skew=('方向', 'skew'), d_sum=('方向', 'sum'))
    new_df['x_max-x_min'] = new_df['x_max'] - new_df['x_min']
    new_df['y_max-y_min'] = new_df['y_max'] - new_df['y_min']
    new_df['x_max-y_min'] = new_df['x_max'] - new_df['y_min']
    new_df['y_max-x_min'] = new_df['y_max'] - new_df['x_min']

    new_df['slope'] = new_df['y_max-y_min'] / np.where(new_df['x_max-x_min']==0, 0.001, new_df['x_max-x_min'])
    new_df['area'] = new_df['x_max-x_min'] * new_df['y_max-y_min']

    new_df['type'] = all_df.groupby('渔船ID').agg(type=('type', 'first'))['type'].values

    X_train = new_df.drop(columns=['type']).iloc[:7000]
    y_train = new_df.iloc[:7000]['type']

    X_test = new_df.drop(columns=['type']).iloc[7000:]

    return X_train, y_train, X_test

In [9]:
def feature_generate_tsfresh():
    train_df = pd.read_csv('./train.csv')
    X_train = train_df.drop(columns=['type'])
    y_train = train_df['type']

    test_df = pd.read_csv('./test.csv')
    X_test = test_df[X_train.columns]
    
    X_train_manully, _, X_test_manully = feature_generate_manually()

    X_train['x_max-x_min'] = X_train_manully['x_max-x_min'].values
    X_test['x_max-x_min'] = X_test_manully['x_max-x_min'].values
    X_train['x_max-y_min'] = X_train_manully['x_max-y_min'].values
    X_test['x_max-y_min'] = X_test_manully['x_max-y_min'].values
    X_train['y_max-x_min'] = X_train_manully['y_max-x_min'].values
    X_test['y_max-x_min'] = X_test_manully['y_max-x_min'].values
    X_train['y_max-y_min'] = X_train_manully['y_max-y_min'].values
    X_test['y_max-y_min'] = X_test_manully['y_max-y_min'].values

    X_train['slope'] = X_train_manully['slope'].values
    X_test['slope'] = X_test_manully['slope'].values
    X_train['area'] = X_train_manully['area'].values
    X_test['area'] = X_test_manully['area'].values

    base_model = lgb.LGBMClassifier(n_estimators=1000, subsample=0.8)
    base_model.fit(X_train.values, y_train)

    selected_columns = X_train.columns[np.argsort(base_model.feature_importances_)[::-1][:60]]
    print(selected_columns)

    X_train = X_train[selected_columns]
    X_test = X_test[selected_columns]
    
    for column in list(X_test.columns[X_test.isnull().sum() > 0]):
        mean_val = X_test[column].mean()
        X_test[column].fillna(mean_val, inplace=True)

    return X_train.values, y_train.values, X_test.values

In [10]:
X_train_tsfresh, y_train, X_test_tsfresh = feature_generate_tsfresh()

Index(['y__quantile__q_0.9', 'y__maximum', 'x__maximum', 'x__abs_energy',
       'y__sum_of_reoccurring_data_points', 'x__quantile__q_0.8',
       'y__quantile__q_0.7', 'y__abs_energy', 'x_max-y_min', 'y_max-x_min',
       'x__sum_of_reoccurring_data_points',
       'y__fft_coefficient__coeff_0__attr_"abs"', 'x__median',
       'x__quantile__q_0.3', '速度__number_crossing_m__m_1',
       '方向__quantile__q_0.9', 'x__quantile__q_0.1', 'x__quantile__q_0.2',
       'x__minimum', 'x__quantile__q_0.9', 'x__sum_of_reoccurring_values',
       '速度__quantile__q_0.7', 'x__quantile__q_0.6',
       '速度__agg_autocorrelation__f_agg_"median"__maxlag_40',
       '方向__ar_coefficient__k_10__coeff_0', 'x__quantile__q_0.7',
       'y__quantile__q_0.8', 'y__sum_of_reoccurring_values',
       '速度__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'x__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_20',
       '速度__ratio_beyond_r_sigma__r_2',
       '速度__time_reversal_asymmetry_statistic__lag_1',
       

In [11]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

In [12]:
X_train = np.concatenate([X_train_tsfresh], axis=1)
X_test = np.concatenate([X_test_tsfresh], axis=1)

In [16]:
X_train = np.concatenate([X_train_tsfresh, np.array(cross_features[:7000])], axis=1)
X_test = np.concatenate([X_test_tsfresh, np.array(cross_features[7000:])], axis=1)

In [17]:
from sklearn.metrics import f1_score

def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [18]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

kf = KFold(n_splits=5, random_state=2020, shuffle=True)

model_list = []
score_list = []
for train_index, test_index in kf.split(X_train):
    model = get_model()
    eval_set = (X_train[test_index], y_train[test_index])
    model.fit(X_train[train_index], y_train[train_index])
    model_list.append(model)
    score_list.append(f1_score(y_train[test_index], model.predict(X_train[test_index]), average='macro'))
    
print(score_list)
print(np.mean(score_list), np.std(score_list))

[0.9146630496021539, 0.9090595606922286, 0.8951638241853672, 0.9156773967728026, 0.9209264607508024]
0.9110980584006709 0.008812857514843628
